In [1]:
from warnings import filterwarnings
filterwarnings("ignore")
from bs4 import BeautifulSoup
import requests

In [2]:
url = "https://www.ndtv.com/latest/page-"

In [3]:
urllist = []
for i in range(1,6,1):
    x = url + str(i)+'/'
    urllist.append(x)

In [4]:
urllist

['https://www.ndtv.com/latest/page-1/',
 'https://www.ndtv.com/latest/page-2/',
 'https://www.ndtv.com/latest/page-3/',
 'https://www.ndtv.com/latest/page-4/',
 'https://www.ndtv.com/latest/page-5/']

# Fetch Headlines and URLs

In [5]:
headlines = []
URLS = []
def fetch_headlines_url(urllist):
    for i in urllist:
        data = requests.get(i)
        soup = BeautifulSoup(data.content,"html.parser")
        for j in soup.find_all("h2",class_=["newsHdng"]):
            headlines.append(j.find("a").text)
        for k in soup.find_all("h2",class_=["newsHdng"]):
            URLS.append(k.find("a")["href"]) 
    return headlines,URLS

In [6]:
Headlines,urls = fetch_headlines_url(urllist)

In [7]:
len(Headlines)

75

In [8]:
urls[:5]

['https://sports.ndtv.com/football/premier-league-newcastle-uniteds-kieran-trippier-faces-spell-on-sidelines-after-breaking-foot-2768145',
 'https://www.ndtv.com/education/jmi-collaborates-germans-diz-train-students-human-wildlife-conflict-mitigation-strategies',
 'https://www.ndtv.com/india-news/hijab-row-nitish-kumar-says-hijab-not-an-issue-in-bihar-we-respect-religious-sentiments-2768014',
 'https://www.ndtv.com/education/swachh-vidyalaya-puraskar-2021-2022-school-can-apply-till-march-end',
 'https://www.ndtv.com/business/coal-india-q3-net-profit-rises-48-to-rs-4-558-cr-2767987']

# Fetch Articles

In [9]:
def Fetch_articles(urls):
    articles = []
    for i in urls:
        data1 = requests.get(i)
        soup1 = BeautifulSoup(data1.content,"html.parser")
        art = ""
        for j in soup1.find_all("p"):
            art = art + str(j.text)
        articles.append(art)
    return articles

In [10]:
Articles = Fetch_articles(urls)

In [11]:
len(Articles)

75

In [13]:
import pandas as pd
Q = pd.DataFrame([headlines,Articles]).T
Q.columns = ["Headline","News_Article"]
Q

,Headline,News_Article
0,Premier League: Newcastle United's Kieran Trip...,Kieran Trippier has been ruled out of Newcastl...
1,JMI Collaborates With German’s DIZ To Train St...,SectionsThe AJK Mass Communication Research Ce...
2,"Hijab Not An Issue In Bihar, We Respect Religi...","Hijab Row: Nitish Kumar said ""we should not fo..."
3,Swachh Vidyalaya Puraskar 2021-2022: School Ca...,SectionsThe Department of School Education and...
4,Coal India December Quarter Profit Rises 48% T...,Coal India reported 48 per cent jump in its th...
...,...,...
70,Yogurt And Strawberry: Nmami Agarwal Shares He...,We love eating and experimenting with food. At...
71,Kareena Kapoor Khan Celebrates Valentine's Day...,Valentine's Day is being celebrated all over t...
72,"Intel's Crypto Mining Accelerator Is 1,000 Tim...",Photo Credit: Intel Corp.​Intel’s blockchain a...
73,"Sensex Crashes 1,747 Points Amid Global Sell-O...",The overall market breadth stood extremely wea...


# Preprocessing

In [30]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
p_art =[]
for i in Articles:
    q = i.upper()
    import re
    q = re.sub("[^A-Z0-9 ]","",q)
    tk_q = q.split(" ")
    sent = ""
    for j in tk_q:
        sent = sent + " " + wnl.lemmatize(ps.stem(j)).upper()
    p_art.append(sent)

# Feature Extraction

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words="english")
X  = tfidf.fit_transform(p_art).toarray()

In [33]:
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.10626616,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.03058763, 0.03058763, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.0268979 , 0.0268979 , ..., 0.        , 0.        ,
        0.04994607]])

# Kmeans Clustering

In [63]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3)
model = km.fit(X)

In [64]:
model.labels_

array([0, 2, 1, 2, 1, 1, 1, 1, 0, 1, 0, 0, 1, 2, 1, 1, 1, 1, 2, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 2, 2, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 2,
       1, 1, 2, 0, 0, 1, 1, 0, 1, 1, 0, 1, 2, 1, 0, 2, 0, 0, 2, 0, 1, 0,
       0, 0, 2, 0, 1, 1, 0, 1, 0])

In [65]:
len(headlines)

75

In [55]:
model.cluster_centers_.argsort()[:,:-41:-1]

array([[2133, 1076, 4480, 3813, 3368, 2129, 4115, 4808, 3970, 4148, 4232,
        2495, 2882, 4330, 3796, 3158, 3990, 1424, 4646, 2636, 2512, 1737,
         390, 5025, 2763, 1843, 1259, 3077, 3397, 2126, 2703,  391, 2282,
        3185, 2690, 1425,  563, 2136, 3754, 5158],
       [4267, 1792, 1857, 2692, 1185, 1811, 1740,  463, 4674,  628,  629,
        2332, 2544, 1229, 3486, 5379, 1370, 5077,  152, 2429, 3583, 1858,
        4824, 1948, 3033, 3499, 3466, 1093,  470, 4801, 3833,   41, 2567,
        1090, 1363, 1973,   29, 2152, 5245, 1044],
       [2639, 5208, 1398, 2348, 4189,  152, 5072, 2959, 2875,  722, 2269,
        4856, 3191, 3343, 3738, 4276, 3280, 3492, 4802, 5310, 1300, 4794,
        3368, 1143, 2495, 2048, 4367, 4517, 1755, 2140, 2212, 2490, 5284,
        2836,  969, 2939, 4922, 3876,  754, 2049]], dtype=int64)

In [56]:
q1 =tfidf.get_feature_names()

In [72]:
#q1

# Top terms per cluster

In [68]:
common_words = model.cluster_centers_.argsort()[:,:-41:-1]
for num,centroid in enumerate(common_words):
    print(num,"     "+ ','.join(q1[word] for word in centroid))

0      ipl,cricket,updat,auction,2022,score,latest,new,android,app,topic,twitter,wi,ind,ha,zoom,mention,ndtv,facebook,download,sport,thi,like,team,ring,schedul,check,smartphon,realm,game,bowl,gadget,mg,redmi,cent,india,australia,hi,oneplu,articl
1      wa,said,minist,hi,ha,tata,live,mr,india,congress,chief,latest,post,new,share,thi,govern,elect,gandhi,kumar,ayci,budget,punjab,polic,cent,health,day,special,case,food,taken,airlin,coverag,2022watch,ndtvcom,newsfollow,advertis,peopl,woman,ukrain
2      school,engin,exam,jee,class,entranc,educ,admiss,student,institut,help,colleg,appli,cours,upje,paper,examin,2022,applic,main,offici,cgso,advanc,teacher,programm,earthquak,test,cet,counsel,inform,gate,websit,career,12,open,held,februari,access,endeavor,path


# Identify Topics

In [59]:
from sklearn.decomposition import NMF
nm = NMF(n_components=3, random_state=31)
model = nm.fit(X)

In [71]:
new_articles = ["ipl 2022 auction"]
q = nm.transform(tfidf.transform(new_articles)).argsort()[:,-1]
print("Following article belongs to cluster name",q)


Following article belongs to cluster name [0]
